In [104]:
import mailparser
import re
import pandas as pd
import io
import glob
import numpy as np

In [428]:

class EmlFileParser(object):
    def __init__(self, file):
        super().__init__()
        self.file = file   

    def parse(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='SITREP|^J\s|^L\s|^N\s')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = df.cross_sitrep.str.extract("(?:MRCC|CROSS|JRCC) ([A-Z]+\/\d+/(?:N°)*\d+)")
        return df
  
    def parse_tahiti(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='JRCC|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = "TAHITI"
        return df
        
        
    def eml_to_text(self):
        mail = mailparser.parse_from_file(self.file)
        return mail.body

    def text_to_df(self):
        a =  self.eml_to_text().split('\n\n')
        return pd.DataFrame(a)



class EmlFolderParser(object):
    def __init__(self, folder, output):
        super().__init__()
        self.folder = folder
        self.output = output
    
    def parse_folder(self):
        text_df = pd.DataFrame()
        for filename in glob.glob(self.folder+'/*.eml'):
            if ('TAHITI' in filename)== True:            
                out = EmlFileParser(filename).parse_tahiti()
            else:
                out = EmlFileParser(filename).parse()
            text_df = text_df.append(out, ignore_index=True)
        text_df.to_csv(self.output)
    

In [429]:
EmlFolderParser('./liste_sitrep', './liste_sitrep/test_all4.csv').parse_folder()

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [313]:
def parse_sitrep(input_path, output_path):
    text_df = pd.DataFrame() 
    for filename in glob.glob(input_path+'/*.eml'):
        mail = mailparser.parse_from_file(filename)
        text = mail.body
        a = text.split('\n\n')
        df = pd.DataFrame(a)
        df.columns =['test']
        df['A'], df['B'] = df['test'].str.split('\n',1).str 
        df = df.drop_duplicates('A', keep='first')
        df1 = df.transpose()
        df1.columns = df1.iloc[1]
        df2 = df1.reindex(df1.index.drop('A')).reset_index()
        df3 = df2.filter(regex='SITREP|^J\s|^L\s|^N\s').drop(0)
        df3.iloc[:,0] =df3.columns[0]
        df3 = df3.rename(columns={ df3.columns[0]: "cross_sitrep"})
        df3['cross_sitrep'] = df3.cross_sitrep.str.extract("(?:MRCC|CROSS|JRCC) ([A-Z]+\/\d+/(?:N°)*\d+)")#ajouter un truc pour La garde 
        text_df = text_df.append(df3, ignore_index=True)
    text_df.to_csv(output_path)


    

In [357]:
parse_sitrep('./liste_sitrep', './liste_sitrep/test_all2.csv')

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [380]:
EmlFileParserTest('./liste_sitrep/JRCC TAHITI - SITREP SAR 415-2017- UN ET FINAL.eml').parse()

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


A,cross_sitrep,J- PREMIERES MESURES PRISES,L- INSTRUCTIONS POUR LA COORDINATION,N- RENSEIGNEMENTS COMPLEMENTAIRES,JRCC TAHITI
0,NaN,14H46 – MISE EN ŒUVRE DE LA VEDETTE DE LA BRIG...,14H48 – CONTRE APPEL AU TEMOIN – APERCOIT UN N...,14H57 – OPERATION CLOSE\n 01 PERSON...,Centre de coordination de sauvetage aéro marit...
